# TEXT SUMMARIZATION 

In [43]:
# DATASET IS : AMAZON REWIEW AVAILABLE IN KAGGLE 

In [1]:
# IMPORT USEFUL LIBRARIES
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
import keras.backend as K

Using TensorFlow backend.


In [2]:
def softmax_over_time(x):
    assert(K.ndim(x) > 2)
    e = K.exp(x - K.max(x,axis=1,keepdims=True))
    s = K.sum(e,axis=1,keepdims=True)
    return e/s

In [3]:
EPOCHS=70
LATENT_DIM=256
EMBEDDING_DIM=100
BATCH_SIZE=64
MAX_NUM_WORDS=40000

In [4]:
# taking only first 50000 reviews
data=pd.read_csv("./amazon-fine-food-reviews/Reviews.csv",nrows=50000)

In [5]:
data.columns=["1","2","3","4","5","6","7","8","target","Text"]

In [6]:
data.head()

,1,2,3,4,5,6,7,8,target,Text
0,500273,B000YUOWVO,A254AOCWGTFNQG,"Andrea ""Andrea""",0,0,5,1235260800,The Best Creamer Ever!!!,The BEST CREAMER EVER!!!! I love this creamer...
1,500274,B000YUOWVO,A1AH8OWVZ85YRL,Joey D. Veasey,1,2,2,1270598400,Good Stuff - but PRICE IS CLIMBING,This is good stuff (some probs with the pump)<...
2,500275,B000YUOWVO,A2UQ16KW38FILX,kelsey13,0,1,5,1336262400,coffee heaven,this is the stuff you can only get at gas stat...
3,500276,B000YUOWVO,A2V1K5WX0VGZI4,Noah D. Goldblatt,2,5,1,1288224000,"No Refrigeration Required, Full of Trans Fat!",I purchased this because I was in a dorm room ...
4,500277,B003QNLUTI,A2EKWSI3QO8WG1,John Morgan,1,1,5,1338249600,Lots of ENERGY,I Work 12 hour shifts that usually turn into 1...


In [7]:
data.drop_duplicates(subset=["Text"],inplace=True)
data.dropna(inplace=True)

In [8]:
data.describe()

,1,5,6,7,8
count,45198.000000,45198.000000,45198.00000,45198.000000,4.519800e+04
mean,524667.827050,1.586398,2.05310,4.192840,1.297111e+09
std,14362.737112,5.156120,5.84521,1.302383,4.780095e+07
min,500273.000000,0.000000,0.00000,1.000000,1.057104e+09
25%,512430.250000,0.000000,0.00000,4.000000,1.272586e+09
50%,524599.500000,0.000000,1.00000,5.000000,1.311638e+09
75%,537317.750000,2.000000,2.00000,5.000000,1.333670e+09
max,550265.000000,281.000000,286.00000,5.000000,1.351210e+09


In [9]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

## Text Cleaning

In [10]:
stopwords=set(stopwords.words("english"))

In [11]:
def text_cleaner(text):
    new_string=text.lower()
    new_string=BeautifulSoup(new_string,'lxml').text  # remove html tags
    new_string = re.sub(r'\([^)]*\)','', new_string)
    new_string = re.sub('"','', new_string)    
    new_string=" ".join([contraction_mapping[t] if t in contraction_mapping else t for t in new_string.split(" ")]) # contraction mapping
    new_string= re.sub("[^a-zA-Z]", " ", new_string) # remove punctuations
    new_string= re.sub(r"'s/b","",new_string)
    
    tokens = [w for w in new_string.split() if not w in stopwords] # stopword removal
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
            
    return (" ".join(long_words)).strip()

cleaned_text=[]
for t in data['Text']:
    cleaned_text.append(text_cleaner(t))
    
    

In [12]:
def summary_cleaner(text):
    new_string=text.lower()
    new_string=re.sub('"','',new_string)
    new_string = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_string.split(" ")])
    new_string= re.sub("[^a-zA-Z]", " ", new_string) # remove punctuations
    new_string= re.sub(r"'s/b","",new_string)
    tokens=new_string.split()
    new_string=''
    for i in tokens:
        if len(i)>1:                                 
            new_string=new_string+i+" " 
    return new_string.rstrip()

cleaned_summary = []
for t in data['target']:
    cleaned_summary.append(summary_cleaner(t))

In [13]:
input_texts = cleaned_text
target_texts = []
target_texts_inputs=[]

for sen in cleaned_summary:
    sen1 = "<sos>" + sen
    sen2 = sen + "<eos>"
    target_texts.append(sen2)
    target_texts_inputs.append(sen1)

In [14]:
print(len(input_texts))
print(len(target_texts))
print(len(target_texts_inputs))


45198
45198
45198


In [15]:
input_texts_1=[]
target_texts_1=[]
target_texts_input_1=[]
for t in range(len(input_texts)):
    if len(target_texts[t].split()) <= 10 and len(input_texts[t].split()) <= 50  :
        input_texts_1.append(input_texts[t])
        target_texts_1.append(target_texts[t])
        target_texts_input_1.append(target_texts_inputs[t])

In [16]:
print(len(input_texts_1))
print(len(target_texts_1))
print(len(target_texts_input_1))

35234
35234
35234


In [17]:
print(target_texts_input_1[5])
print(target_texts_1[5])

<sos>lives up to its claims
lives up to its claims<eos>


In [18]:
# Tokenize input sentence
NUM_OF_SAMPLES = len(input_texts_1)
tokenizer_input = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_input.fit_on_texts(input_texts_1)
input_sequences = tokenizer_input.texts_to_sequences(input_texts_1)

# Tokenize encoder input and output
tokenizer_output = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_output.fit_on_texts(target_texts_input_1+["<eos>"])
target_sequences_input = tokenizer_output.texts_to_sequences(target_texts_input_1)
target_sequences = tokenizer_output.texts_to_sequences(target_texts_1)

In [19]:
print(NUM_OF_SAMPLES)

35234


In [20]:
print(target_sequences_input[0])
print(target_sequences[0])

[1, 4, 6, 1676, 32]
[4, 6, 1676, 32, 3228]


In [21]:
word2idx_input=tokenizer_input.word_index
word2idx_output=tokenizer_output.word_index

In [22]:
word2idx_output["eos"]
print(len(target_sequences))
print(len(target_sequences_input))


35234
35234


In [23]:
print("Unique words: ",len(word2idx_input))

Unique words:  26041


In [24]:
print(word2idx_output["sos"])
num_words_output = len(word2idx_output)+1
print(num_words_output)

1
8618


In [25]:
max_len_input=max(len(a) for a in input_sequences)
print(max_len_input)
# 1029 is a big values

50


In [26]:
max_len_target=max(len(a) for a in target_sequences)
print(max_len_target)

11


In [27]:
# Padding of sequences for encoder
encoder_inputs = pad_sequences(input_sequences, maxlen=max_len_input)
print("encoder_inputs.shape:", encoder_inputs.shape)
print("encoder_inputs[0]:", encoder_inputs[0])

encoder_inputs.shape: (35234, 50)
encoder_inputs[0]: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0   15  895   68    6  895   18
    5   80 1854    4   72  213  615   19   13  416  115   13    2   95
   34  335  279   18  741  895   55   13]


In [28]:
# padding of sequences for decoder
decoder_inputs = pad_sequences(target_sequences_input, maxlen=max_len_target,padding='post')
print("decoder_inputs.shape:", decoder_inputs.shape)
print("decoder_inputs[0]:", decoder_inputs[0])

decoder_inputs.shape: (35234, 11)
decoder_inputs[0]: [   1    4    6 1676   32    0    0    0    0    0    0]


In [29]:
# padding of sequences for decoder - Teacher forcing
decoder_targets =pad_sequences(target_sequences,maxlen=max_len_target,padding='post')
print("decoder_targets.shape:", decoder_targets.shape)
print("decoder_targets[0]:", decoder_targets[0])

decoder_targets.shape: (35234, 11)
decoder_targets[0]: [   4    6 1676   32 3228    0    0    0    0    0    0]


In [30]:
# create embedding matrix
word2vec = {}
with open("./glove6b100dtxt/glove.6B.100d.txt",encoding='utf8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vector=np.asarray((values[1:]),dtype='float32')
        word2vec[word]=vector


In [31]:
# PREPARE EMBEDDING MATRIX
num_words = min(MAX_NUM_WORDS,len(word2idx_input)+1)
embedding_matrix = np.zeros((num_words,EMBEDDING_DIM))
for word, i in word2idx_input.items():
    if i < MAX_NUM_WORDS:
        embedding_vector=word2vec.get(word)
        
        if embedding_vector is not None:
            embedding_matrix[i]=embedding_vector


In [32]:
decoder_targets_onehot=to_categorical(decoder_targets)
print(decoder_targets_onehot.shape)

(35234, 11, 8618)


# Model Building

In [33]:
embedding_layer=Embedding(num_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_len_input)

In [34]:
# SETTING UP ENCODER
encoder_inputs_placeholder = Input(shape=(max_len_input,))
x = embedding_layer(encoder_inputs_placeholder)
encoder = Bidirectional(LSTM(
  LATENT_DIM,
  return_sequences=True,
  dropout=0.5 # dropout not available on gpu
))
encoder_outputs = encoder(x)

decoder_inputs_placeholder = Input(shape=(max_len_target,))

decoder_embedding = Embedding(num_words_output, EMBEDDING_DIM)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)




######### Attention #########
# Attention layers need to be global because
# they will be repeated Ty times at the decoder
attn_repeat_layer = RepeatVector(max_len_input)
attn_concat_layer = Concatenate(axis=-1)
attn_dense1 = Dense(10, activation='tanh')
attn_dense2 = Dense(1, activation=softmax_over_time)
attn_dot = Dot(axes=1) # to perform the weighted sum of alpha[t] * h[t]

def one_step_attention(h, st_1):
    # h = h(1), ..., h(Tx), shape = (Tx, LATENT_DIM * 2)
    # st_1 = s(t-1), shape = (LATENT_DIM_DECODER,)

    # copy s(t-1) Tx times
    # now shape = (Tx, LATENT_DIM_DECODER)
    st_1 = attn_repeat_layer(st_1)

    # Concatenate all h(t)'s with s(t-1)
    # Now of shape (Tx, LATENT_DIM_DECODER + LATENT_DIM * 2)
    x = attn_concat_layer([h, st_1])

    # Neural net first layer
    x = attn_dense1(x)
    # Neural net second layer with special softmax over time
    alphas = attn_dense2(x)

    # "Dot" the alphas and the h's
    # Remember a.dot(b) = sum over a[t] * b[t]
    context = attn_dot([alphas, h])
    

    return context


# define the rest of the decoder (after attention)
decoder_lstm = LSTM(LATENT_DIM, return_state=True)
decoder_dense = Dense(num_words_output, activation='softmax')

initial_s = Input(shape=(LATENT_DIM,), name='s0')
initial_c = Input(shape=(LATENT_DIM,), name='c0')
context_last_word_concat_layer = Concatenate(axis=2)



s = initial_s
c = initial_c

# collect outputs in a list at first
outputs = []
for t in range(max_len_target): # Ty times
    # get the context using attention
    context = one_step_attention(encoder_outputs, s)

    # we need a different layer for each time step
    selector = Lambda(lambda x: x[:, t:t+1]) 
    
    xt = selector(decoder_inputs_x)
    # combine 
    decoder_lstm_input = context_last_word_concat_layer([context, xt])

    # pass the combined [context, last word] into the LSTM
    # along with [s, c]
    # get the new [s, c] and output
    o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[s, c])
    # final dense layer to get next word prediction
    decoder_outputs = decoder_dense(o)

    outputs.append(decoder_outputs)


# 'outputs' is now a list of length Ty
# each element is of shape (batch size, output vocab size)
# therefore if we simply stack all the outputs into 1 tensor
# it would be of shape T x N x D
# we would like it to be of shape N x T x D

def stack_and_transpose(x):
    # x is a list of length T, each element is a batch_size x output_vocab_size tensor
    x = K.stack(x) # is now T x batch_size x output_vocab_size tensor
    x = K.permute_dimensions(x, pattern=(1, 0, 2)) # is now batch_size x T x output_vocab_size
    return x

# make it a layer
stacker = Lambda(stack_and_transpose)
outputs = stacker(outputs)
print(outputs.shape)

# create the model
model = Model(
  inputs=[
    encoder_inputs_placeholder,
    decoder_inputs_placeholder,
    initial_s, 
    initial_c,
  ],
  outputs=outputs
)


(None, 11, 8618)


In [35]:
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 100)      2604200     input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 [(None, 256)]        0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 50, 512)      731136      embedding[0][0]                  
______________________________________________________________________________________________

In [36]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1)

In [37]:
"""z = np.zeros((len(encoder_inputs), LATENT_DIM)) # initial [s, c]
r = model.fit(
  [encoder_inputs, decoder_inputs, z, z], decoder_targets_onehot,
  batch_size=BATCH_SIZE,
  epochs=20,
  callbacks=[es]
)"""

'z = np.zeros((len(encoder_inputs), LATENT_DIM)) # initial [s, c]\nr = model.fit(\n  [encoder_inputs, decoder_inputs, z, z], decoder_targets_onehot,\n  batch_size=BATCH_SIZE,\n  epochs=20,\n  callbacks=[es]\n)'

In [38]:
 model.load_weights("./new_weights1.h5")

# MODEl FOR PREDICTION

In [39]:
encoder_model = Model(encoder_inputs_placeholder, encoder_outputs)

encoder_outputs_as_input = Input(shape=(max_len_input, LATENT_DIM * 2,))
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)


context = one_step_attention(encoder_outputs_as_input, initial_s)
decoder_lstm_input = context_last_word_concat_layer([context, decoder_inputs_single_x])


o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
decoder_outputs = decoder_dense(o)

decoder_model = Model(
  inputs=[
    decoder_inputs_single,
    encoder_outputs_as_input,
    initial_s, 
    initial_c
  ],
  outputs=[decoder_outputs, s, c]
)

In [40]:
idx2word_eng = {v:k for k, v in word2idx_input.items()}
idx2word_trans = {v:k for k, v in word2idx_output.items()}

In [41]:
def decode_sequence(input_seq):
    
    enc_out = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))

    target_seq[0, 0] = word2idx_output['sos']

    
    eos = word2idx_output['eos']

    s = np.zeros((1, LATENT_DIM))
    c = np.zeros((1, LATENT_DIM))

    output_sentence = []
    for _ in range(max_len_target):
        o, s, c = decoder_model.predict([target_seq, enc_out, s, c])
        idx = np.argmax(o.flatten())

        # End sentence of EOS
        if eos == idx:
            break

        word = ''
        if idx > 0:
            word = idx2word_trans[idx]
            output_sentence.append(word)

        # Update the decoder input
        # which is just the word just generated
        target_seq[0, 0] = idx

    return ' '.join(output_sentence)


In [42]:
while True:
    # Do some test translations
    i = np.random.choice(len(input_texts_1))
    input_seq = encoder_inputs[i:i+1]
    translation_a = decode_sequence(input_seq)
    print('-')
    print('Input:', input_texts_1[i])
    print('Translation:', translation_a)
    print('Original:',target_texts_1[i])

    ans = input("Continue? [Y/n]")
    if ans and ans.lower().startswith('n'):
        break

-
Input: bags seem work well feel sturdy quality materials reviewers commented work measure milk need know much putting ahead time easy adjust far used freeze milk defrosted yet
Translation: very good product
Original: very good product<eos>
Continue? [Y/n]y
-
Input: ordered food tues received next day supplier thank great expeditious customer service
Translation: pinnacle trout sweet
Original: pinnacle trout sweet<eos>
Continue? [Y/n]y
-
Input: live new orleans known deliciour pralines coconut pralines bought kelly second time month best ever fresh right size could eaten one day rationed one day intend buy
Translation: best pralines ever
Original: best pralines ever<eos>
Continue? [Y/n]n


In [0]:
n